# 2CS SIQ2-SIL2 TP04. Naïve Bayes

Dans ce TP, nous allons traiter Naïve Bayes. C'est le seul algorithme dans notre programme qui crée un modèle génératif (en plus des auto-encodeurs).

- **Binôme 01** : 
- **Binôme 02** :
- **Groupe** : SIQ2|SIL2

In [1]:
import matplotlib
import numpy             as np
import pandas            as pd 
import matplotlib.pyplot as plt 
%matplotlib inline

np        .__version__ , \
pd        .__version__ , \
matplotlib.__version__

('1.21.5', '1.4.2', '3.5.1')

In [2]:
from typing import Tuple, List, Dict

**RAPPEL**

Tout le monde connait le théorème de Bayes pour calculer la probabilité conditionnelle d'un évennement $A$ sachant un autre $B$ (si vous ne le connaissez pas; vous n'appartenez pas à tout le monde) : 
$$ P(A|B) = \frac{P(A)P(B|A)}{P(B)}$$

Pour appliquer ce théorème sur un problème d'appentissage automatique, l'idée est simple ; Etant donné une caractéristique $f$ et la sortie $y$ qui peut avoir la classe $c$ : 
- Remplacer $A$ par $y=c$
- Remplacer $B$ par $f$ 
On aura l'équation : 
$$ P(y=c|f) = \frac{P(y=c)P(f|y=c)}{P(f)}$$

On appelle : 
- $P(y=c|f)$ postérieure 
- $P(y=c)$ antérieure #la probabilité de la classe 𝑐 avant l'observation de la 
caractéristique 𝑓. Elle peut être estimée à partir des données d'entraînement en calculant simplement la proportion d'occurrences de chaque classe.
- $P(f|y=c)$ vraisemblance #représente la probabilité que les caractéristiques observées d'un fruit soient telles que le fruit appartienne à une certaine classe (par exemple, une pomme ou une orange).par exemple, la probabilité que la couleur soit rouge sachant que le fruit est une pomme calculer à partir des données de notre ensemble d'entraînement (calculé pour chaque caractéristique).
- $P(f)$ évidence  #représente la probabilité de voir les caractéristiques observées dans notre ensemble de données, sans prendre en compte la classe des fruits. Dans notre exemple de fruits, l'évidence serait la probabilité de voir un fruit avec une certaine couleur et une certaine forme, quelle que soit sa classe.

Ici, on estime la probablité d'une classe $c$ sachant une caractéristique $f$ en utilisant des données d'entrainement. Maintenant, on veut estimer la probabilité d'une classe $c$ sachant un vecteur de caractéristiques $\overrightarrow{f} = \{f_1, ..., f_L\}$ : 
$$ P(y=c|\overrightarrow{f}) = \frac{P(y=c)P(\overrightarrow{f}|y=c)}{P(f)}$$

Etant donnée plusieurs classes $c_j$, la classe choisie $\hat{c}$ est celle avec la probabilité maximale 
$$\hat{c} = \arg\max\limits_{c_k} P(y=c_k|\overrightarrow{f})$$
$$\hat{c} = \arg\max\limits_{c_k} \frac{P(y=c_k)P(\overrightarrow{f}|y=c_k)}{P(f)}$$
On supprime l'évidence pour cacher le crime : $P(f)$ ne dépend pas de $c_k$ et elle est postive, donc ça ne va pas affecter la fonction $\max$.
$$\hat{c} = \arg\max\limits_{c_k} P(y=c_k)P(\overrightarrow{f}|y=c_k)$$

Pour calculer $P(\overrightarrow{f}|y=c_k)$, on va utiliser une properiété naïve (d'où vient le nom Naive Bayes) : on suppose l'indépendence conditionnelle entre les caractéristiques $f_j$. 
$$\hat{c} = \arg\max\limits_{c_k} P(y=c_k) \prod\limits_{f_j \in \overrightarrow{f}} P(f_j|y=c_k)$$

Pour éviter la disparition de la probabilité (multiplication et représentation de virgule flottante sur machine), on transforme vers l'espace logarithme.
$$\hat{c} = \arg\max\limits_{c_k} \log P(y=c_k) + \sum\limits_{f_j \in \overrightarrow{f}} \log P(f_j|y=c_k)$$


## I. Réalisation des algorithmes

Pour estimer la vraisemblance, il existe plusieurs modèles (lois):
- **Loi multinomiale :** pour les caracétristiques nominales
- **Loi de Bernoulli :** lorsqu'on est interressé par l'apparence d'une caractéristique ou non (binaire)
- **Loi normale :** pour les caractéristiques numériques

Dans ce TP, nous allons implémenter Naive Bayes pour les caractéristiques nominales (loi multinomiale). 
Dans notre modèle, nous voulons stocker les statistiques et pas les probabilités. 
L'intérêt est de faciliter la mise à jours des statistiques (si par exemple, nous avons un autre dataset et nous voulons enrichir le modèle ; dans e cas, il suffit d'ajouter les statistiques du nouveau dataset)

Ici, nous allons utiliser le dataset "jouer" (utilisé dans la plupart des cours) contenant des caractéristiques nominales.

In [3]:
jouer   = pd.read_csv('Desktop/2Sem/ML/TPs/TP04/TP04/data/jouer.csv')

X_jouer = jouer.iloc[:, :-1].values # Premières colonnes 
Y_jouer = jouer.iloc[:,  -1].values # Dernière colonne 

# Afficher le dataset "jouer"
jouer

,temps,temperature,humidite,vent,jouer
0,ensoleile,chaude,haute,non,non
1,ensoleile,chaude,haute,oui,non
2,nuageux,chaude,haute,non,oui
3,pluvieux,douce,haute,non,oui
4,pluvieux,fraiche,normale,non,oui
5,pluvieux,fraiche,normale,oui,non
6,nuageux,fraiche,normale,oui,oui
7,ensoleile,douce,haute,non,non
8,ensoleile,fraiche,normale,non,oui
9,pluvieux,douce,normale,non,oui


### I.1. Entraînement de la probabilité antérieure

Etant donné le vecteur de sortie $Y$, la probabilité de chaque classe "oui ou bien non" (différentes valeurs de $Y$) est calulée comme :

$$p(c_k) = \frac{|\{y / y \in Y \text{ et } y = c_k\}|}{|Y|}$$


La fonction doit récupérer des statistiques afin de pouvoir calculer la probabilité antérieure de chaque classe. Donc, elle doit retourner  :
- Un vecteur contenant les noms des classes
- Un vecteur contenant les nombres d'occurrences de chaque classe dans le premier vecteur

In [4]:
# TODO: Stastistiques sur la probabilité antérieure
def stat_anterieure(Y: np.ndarray) -> Tuple[np.ndarray, np.ndarray]: 
    cls  = np.unique(Y) # le vecteur des classes
    freq = []
    for c in cls:
        freq_c = len(Y[Y == c])
        freq.append(freq_c) 
    return cls, np.array(freq)

#=====================================================================
# TEST UNITAIRE
#=====================================================================
# Resultat : 
# (array(['non', 'oui'], dtype=object), array([5, 9]))
#---------------------------------------------------------------------

stat_anterieure(Y_jouer)

(array(['non', 'oui'], dtype=object), array([5, 9]))

### I.2. Entraînement de la probabilité de vraissemblance (loi multinomiale)

Notre modèle doit garder le nombre des différentes valeurs d'une caractéristique $A$ et le nombre de ces valeurs dans chaque classe.
Donc, étant donné un vecteur d'une caractéristique $A= X[:,j]$, un autre des $Y$ et un $C$ contenant la liste des classes, la fonction d'entraînement doit retourner : 
- $V$ : un vecteur contenant les différentes catégories de $A$ (c'est déjà fait)
- Une matrice contenant le nombre d'occurrences de chaque catégorie de $V$ dans chaque classe  : 
   - Les lignes représentent les catégories $v \in V$ de la caréctéristique $A$
   - Les colonnes représentent les classes $c \in C$ de $Y$


In [5]:
# TODO: Statistiques de vraissemblance (une seule caractéristique)
def stat_vraissemblance_1(A: np.ndarray, 
                          Y: np.ndarray, 
                          C: np.ndarray
                         ) -> Tuple[np.ndarray, np.ndarray]: 
    V = np.unique(A) # Catégories de la caractéristique A
    freq = np.zeros((len(V), len(C)))  # Initialize freq with zeros
    for j, c in enumerate(C):
        for i, v in enumerate(V):
            freq_c_v = len(A[(Y == c) & (A == v)]) # Fréquence de la catégorie v de la caractéristique A pour la classe c
            freq[i, j] = freq_c_v
    return V, freq


#=====================================================================
# TEST UNITAIRE
#=====================================================================
# Resultat : 
# (array(['ensoleile', 'nuageux', 'pluvieux'], dtype=object),
#  array([[3, 2],
#         [0, 4],
#         [2, 3]]))
#---------------------------------------------------------------------
C_t = np.array(['non', 'oui'])
stat_vraissemblance_1(X_jouer[:, 0], Y_jouer, C_t)

(array(['ensoleile', 'nuageux', 'pluvieux'], dtype=object),
 array([[3., 2.],
        [0., 4.],
        [2., 3.]]))

### I.3. Entraînement loi multinomiale

**Rien à programmer ici**

Notre modèle ($\theta_{X, C}$) doit garder des statistiques sur les classes et aussi sur chaque catégorie de chaque caractéristique. Pour ce faire, nous allons représenter $\theta$ comme un vecteur : 
- $\theta[N+1]$ est un vecteur de $N$ éléments représentant des statistiques sur chaque caractéristique $j$, plus un élément (le dernier) pour les statistiques sur les classes.
- Chaque élément est un dictionnaire (HashMap en Java)
- Un élément des caractéristiques contient deux clés : 
    - **val** : pour récupérer la liste des noms des catégories de la caractéristique
    - **freq**: pour récupérer une matrice représentant la fréquence de chaque caractéristique dans chaque classe
- Un élément des classes contient deux clés : 
    - **cls** : pour récupérer la liste des noms des classes
    - **freq**: pour récupérer la liste des fréquences de chaque classe

In [7]:
# La fonction qui entraine Théta sur plusieurs caractéristiques
# Rien à programmer ici
# Notre théta est une liste des dictionnaires;
# chaque dictionnaire contient la liste des catégories et la matrice des fréquences dela caractéristique respective à la colonne de X
# On ajoute les statistiques antérieures des classes à la fin de résultat
def entrainer_multi(X: np.ndarray, 
                    Y: np.ndarray
                   ) -> np.ndarray: 
    
    Theta   = []
    
    stats_c = {}
    stats_c['cls'], stats_c['freq'] =  stat_anterieure(Y)
    
    for j in range(X.shape[1]): 
        stats = {}
        stats['val'], stats['freq'] =  stat_vraissemblance_1(X[:, j], Y, stats_c['cls'])
        Theta.append(stats)
    
    Theta.append(stats_c)
    return Theta


#=====================================================================
# TEST UNITAIRE
#=====================================================================
# Resultat : 
# [{'val': array(['ensoleile', 'nuageux', 'pluvieux'], dtype=object),
#   'freq': array([[3, 2],
#          [0, 4],
#          [2, 3]])},
#  {'val': array(['chaude', 'douce', 'fraiche'], dtype=object),
#   'freq': array([[2, 2],
#          [2, 4],
#          [1, 3]])},
#  {'val': array(['haute', 'normale'], dtype=object),
#   'freq': array([[4, 3],
#          [1, 6]])},
#  {'val': array(['non', 'oui'], dtype=object),
#   'freq': array([[2, 6],
#          [3, 3]])},
#  {'cls': array(['non', 'oui'], dtype=object), 'freq': array([5, 9])}]
#---------------------------------------------------------------------
Theta_jouer = entrainer_multi(X_jouer, Y_jouer)

Theta_jouer

[{'val': array(['ensoleile', 'nuageux', 'pluvieux'], dtype=object),
  'freq': array([[3., 2.],
         [0., 4.],
         [2., 3.]])},
 {'val': array(['chaude', 'douce', 'fraiche'], dtype=object),
  'freq': array([[2., 2.],
         [2., 4.],
         [1., 3.]])},
 {'val': array(['haute', 'normale'], dtype=object),
  'freq': array([[4., 3.],
         [1., 6.]])},
 {'val': array(['non', 'oui'], dtype=object),
  'freq': array([[2., 6.],
         [3., 3.]])},
 {'cls': array(['non', 'oui'], dtype=object), 'freq': array([5, 9])}]

### I.4. Estimation de la probabilité de vraissemblance (loi multinomiale)
L'équation pour estimer la vraisemblance 
$$ P(X_j=v|y=c_k) = \frac{|\{ y \in Y / y = c_k \text{ et } X_j = v\}|}{|\{y = c_k\}|}$$


Dans le cas d'une valeur $v$ qui n'existe pas dans le dataset d'entrainnement ou qui n'existe pas pour une classe donnée mais ui existe dans le dataset de test, nous aurons une probabilité nulle. 
Afin de régler ce problème, nous pouvons appliquer une fonction de lissage qui attribue une petite probabilité aux données non vues dans l'entraînement. 
Le lissage que nous allons utiliser est celui de Lidstone. 
Lorsque $\alpha = 1$, il est appelé lissage de Laplace.
$$ P(X_j=v|y=c_k) = \frac{|\{ y \in Y / y = c_k \text{ et } X_j = v\}| + \alpha}{|\{y = c_k\}| + \alpha * |V|}$$
Où: 
- $\alpha$ est une valeur donnée 
- $V$ est l'ensemble des différentes valeurs de $f_j$ (le vocabulaire; les catégories)

Etant donné : 
- $\theta_j$ les paramètres de la caractéristique $j$ représentées comme dictionnaire
    - **val** : pour récupérer la liste des noms des catégories de la caractéristique (vocabulaire $V$)
    - **freq**: pour récupérer une matrice représentant la fréquence de chaque caractéristique dans chaque classe. C'est une matrice $|V|\times|C|$
- $v$ la valeur de la caractéristique $j$ utilisée pour calculer les probabilités
- $\theta_c$ les paramètres des classes $C$ représentées comme dictionnaire
    - **cls** : pour récupérer la liste des noms des classes
    - **freq**: pour récupérer la liste des fréquences des classes
    
Cette fonction doit retourner : 
- Une liste $P[|C|]$ contenant les probabilités de la catégorie $v$ de $X_j$ sur toutes les classes $C$ 
- Elle doit prendre en considération le cas où la valeur $v$ n'existe pas dans le modèle entraîné

In [126]:
def P_vraiss_multi(Theta_j: Dict[str, np.ndarray], 
                   Theta_c: Dict[str, np.ndarray], 
                   v: str, 
                   alpha: float = 0.
                  ) -> np.ndarray:
        
    # une liste des indices où se trouve la valeur v dans Theta_j["val"]
    ind = np.where(Theta_j['val'] == v)[0] 
    
    # Get the vocabulary and frequency matrix of feature j
    vocab_j = Theta_j["val"] 
    freq_j = Theta_j["freq"]
    
    # Get the classes their frequency
    classes = Theta_c["cls"]
    freq_c = Theta_c["freq"]
    
    if len(ind) == 0:
         default_prob = alpha / (freq_c + alpha * len(vocab_j))
         return default_prob
    
    prob_v = (freq_j[ind,:]  / freq_c )[0]
    
    return prob_v


#=====================================================================
# TEST UNITAIRE
#=====================================================================
# Resultat : 
# (array([0.4       , 0.33333333]), array([0.125     , 0.08333333]))
#---------------------------------------------------------------------
# Calcul :
# La probabilité de jouer si temps = pluvieux 
# P(temps = pluvieux | jouer=oui) = (nbr(temps=pluvieux et jouer=oui)+alpha)/(nbr(jour=oui) + alpha * nbr_diff(temps)))
# P(temps = pluvieux | jouer=oui) = (3 + 0)/(9 + 0) ==> 3 est le nombre de différentes valeurs de temps (entrainnement)
# P(temps = pluvieux | jouer=oui) = 4/12 ==> 0.33333333333333333333333333333333333~

# La probabilité de jouer si temps = neigeux 
# P(temps = neigeux | jouer=oui) = (nbr(temps=neigeux et jouer=oui)+alpha)/(nbr(jouer=oui) + alpha * nbr_diff(temps)))
# P(temps = neigeux | jouer=oui) = (0 + 1)/(9 + 3) ==> 3 est le nombre de différentes valeurs de temps (entrainnement)
# P(temps = neigeux | jouer=oui) = 1/13 ==> 0.0833333333333333333333333333333333333~
#---------------------------------------------------------------------

P_vraiss_multi(Theta_jouer[0], Theta_jouer[-1], 'pluvieux'), \
P_vraiss_multi(Theta_jouer[0], Theta_jouer[-1], 'neigeux', alpha=1.)

(array([0.4       , 0.33333333]), array([0.125     , 0.08333333]))

### I.5. Prédiction de la classe (loi multinomiale)
Revenons maintenant à notre équation de prédiction 
$$\hat{c} = \arg\max\limits_{c_k} \log P(y=c_k) + \sum\limits_{f_j \in \overrightarrow{f}} \log P(f_j|y=c_k)$$

- On doit prédire un seule échantillon $x$. 
- La fonction doit retourner un vecteur des log-probabilité des classes
- Si anter=false donc on n'utilise pas la probabilité antérieure

In [149]:
# TODO: Prédiction des log des probabilités
def predire(x: List[str], Theta: List[Dict[str, np.ndarray]], alpha: float = 1., anter: bool = True) -> np.ndarray:
    # Get the number of classes
    K = Theta[-1]["cls"].size

    # Initialize the array of log-probabilities
    log_probs = np.zeros(K)

    # Calculate the log-probability of each class k
    log_prob_k = np.log(Theta[-1]["freq"]) if anter else 0.

    # Calculate the sum of log-probabilities of features for class k
    for k in range(K):
        prob_xj = 0

        for j in range(len(x)):
            # Get the vocabulary and frequency matrix of feature j 
            Theta_j = Theta[j]

            # Get the number of categories for feature j
            J = Theta_j["val"].shape[0]

            # Calculate the probability of feature x_j given class k
            prob_xj += np.log(P_vraiss_multi(Theta_j, Theta[-1], x[j], alpha)[k])

        # Get the frequency of class k
        freq_k = Theta[-1]["freq"][k]

        # Calculate the log-probability of class k given feature x
        log_probs[k] = log_prob_k + np.log(freq_k) + prob_xj

    return log_probs
# TEST UNITAIRE
#=====================================================================
# Resultat : 
# (array([-5.20912179, -4.10264337]), array([-4.17950237, -3.66081061]))
#---------------------------------------------------------------------
predire(['pluvieux', 'fraiche', 'normale', 'oui'], Theta_jouer), \
predire(['pluvieux', 'fraiche', 'normale', 'oui'], Theta_jouer, anter=False) 

ValueError: setting an array element with a sequence.

### I.6. Regrouper en une classe (loi multinomiale)

**Rien à programmer ici**


In [9]:
class NBMultinom(object): 
    
    def __init__(self, alpha=1.): 
        self.alpha = alpha
        
    def entrainer(self, X, Y):
        self.Theta = entrainer_multi(X, Y)
    
    def predire(self, X, anter=True, prob=False): 
        Y_pred = []
        cls = self.Theta[-1]['cls']
        for i in range(len(X)): 
            log_prob = predire(X[i,:], self.Theta, alpha=self.alpha, anter=anter)
            if prob:
                Y_pred.append(np.max(log_prob))
            else:
                Y_pred.append(cls[np.argmax(log_prob)])
        return Y_pred

#=====================================================================
# TEST UNITAIRE
#=====================================================================
# Resultat : 
# ['oui', 'non']
#---------------------------------------------------------------------
notre_modele = NBMultinom()
notre_modele.entrainer(X_jouer, Y_jouer)
X_test = np.array([
    ['neigeux', 'fraiche', 'normale', 'oui'],
    ['neigeux', 'fraiche', 'haute'  , 'oui']
])
notre_modele.predire(X_test)

['oui', 'non']

## II. Application et analyse

**Il n'y a rien à programmer ici.**

Le but de cette section est de mener des expérimentations afin de bien comprendre les concepts vus dans le cours.
Aussi, elle nous assiste à comprendre l'effet des différents paramètres.
En plus, la discussion des différentes expérimentations peut améliorer l'aspect analytique chez l'étudient.

### II.1. Probabilité antérieure 

Nous voulons tester l'effet de la probabilité antérieure.
Pour ce faire, nous avons entraîné deux modèles :
1. Avec probabilité antérieure
1. Sans probabilité antérieure (Il considère une distribution uniforme des classes)

Pour tester si les modèles ont bien s'adapter au dataset d'entraînement, nous allons les tester sur le même dataset et calculer le rapport de classification.


In [150]:
# AVEC Scikit-learn
# ===================
from sklearn.naive_bayes   import CategoricalNB
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics       import classification_report

nb_avec     = CategoricalNB(alpha=1.0, fit_prior=True )
nb_sans     = CategoricalNB(alpha=1.0, fit_prior=False)

enc         = OrdinalEncoder()
X_jouer_enc = enc.fit_transform(X_jouer)
nb_avec.fit(X_jouer_enc, Y_jouer)
nb_sans.fit(X_jouer_enc, Y_jouer)

Y_pred_avec = nb_avec.predict(X_jouer_enc)
Y_pred_sans = nb_sans.predict(X_jouer_enc)

# AVEC notre modèle (juste pour voir comment l'utiliser)
# =======================================================
#notre_modele = NBMultinom()
#notre_modele.entrainer(X_jouer, Y_jouer)
#Y_notre_ant = notre_modele.predire(X_jouer)
#Y_notre_sans_ant = notre_modele.predire(X_jouer, anter=False) 

# Le rapport de classification


print( 'Avec probabilité antérieure (a priori)'  )
print(classification_report(Y_jouer, Y_pred_avec))

print( 'Sans probabilité antérieure (a priori)'  )
print(classification_report(Y_jouer, Y_pred_sans))

Avec probabilité antérieure (a priori)
              precision    recall  f1-score   support

         non       1.00      0.80      0.89         5
         oui       0.90      1.00      0.95         9

    accuracy                           0.93        14
   macro avg       0.95      0.90      0.92        14
weighted avg       0.94      0.93      0.93        14

Sans probabilité antérieure (a priori)
              precision    recall  f1-score   support

         non       0.67      0.80      0.73         5
         oui       0.88      0.78      0.82         9

    accuracy                           0.79        14
   macro avg       0.77      0.79      0.78        14
weighted avg       0.80      0.79      0.79        14



**TODO: Analyser les résultats**
    
- Que remarquez-vous ?
- Est-ce que la probabilité antérieure est importante dans ce cas ?
- Comment cette probabilité affecte le résultat ?
- Quand est-ce que nous sommes sûrs que l'utilisation de cette probabilité est inutile ?

**Réponse**

- En analysant les résultats, on peut remarquer que la précision, le rappel et le score F1  et l'accuracy globale et meme les valeurs de macro avg et weighted avg sont meilleurs pour le modèle utilisant la probabilité antérieure que pour celui sans probabilité antérieure.

- Oui, la probabilité antérieure est importante dans ce cas. En effet, sans la probabilité antérieure, le modèle considère une distribution uniforme des classes, ce qui est un mauvais choix dans notre cas car les classes ne sont pas équitablement représentées dans ce jeu de données (5 pour la classe "non" et 9 pour la classe "oui") ,Dans notre exemple, Lorsque nous utilisons la probabilité antérieure, le modèle prend en compte ces fréquences dans la phase d'apprentissage et ajuste les paramètres en conséquence. Ainsi, le modèle sera mieux équipé pour traiter les cas où les classes sont déséquilibrées,De plus le fait que le nombre d'exemples pour chaque classe soit petit peut rendre difficile l'apprentissage des modèles sans la probabilité antérieure.

- Lorsque nous utilisons la probabilité antérieure (a priori), le modèle suppose que toutes les classes ont une probabilité différente, en fonction de leur fréquence dans l'ensemble des données d'entraînement. En revanche, lorsque nous n'utilisons pas la probabilité antérieure, le modèle suppose que toutes les classes ont une probabilité égale .Donc, l'effet de la probabilité antérieure dépend de la distribution des classes dans l'ensemble des données d'entraînement. Si une classe est sous-représentée dans l'ensemble des données d'entraînement, l'utilisation de la probabilité antérieure peut aider à corriger cette disproportion en augmentant la probabilité de cette classe. Si toutes les classes sont représentées de manière équivalente dans l'ensemble des données d'entraînement, l'utilisation de la probabilité antérieure peut ne pas avoir beaucoup d'impact sur les estimations de probabilité du modèle.

- En général, l'utilisation de la probabilité antérieure peut être inutile lorsque les classes sont équitablement représentées dans le jeu de données, c'est-à-dire lorsque chaque classe a approximativement la même proportion d'exemples. Dans ce cas, l'utilisation de la probabilité antérieure n'apportera pas de gain significatif en termes de performances et pourrait même causer une baisse de celles-ci si la probabilité antérieure choisie est mal adaptée.

### II.2. Lissage

Nous voulons tester l'effet de lissage de Lidstone.
Pour ce faire, nous avons entraîné trois modèles : 
1. alpha = 1 (lissage de Laplace)
1. alpha = 0.5
1. alpha = 0 (sans lissage)

In [151]:
NBC_10 = CategoricalNB(alpha = 1.0 )
NBC_05 = CategoricalNB(alpha = 0.5 )
NBC_00 = CategoricalNB(alpha = 0.0 )

NBC_10.fit( X_jouer_enc,   Y_jouer )
NBC_05.fit( X_jouer_enc,   Y_jouer )
NBC_00.fit( X_jouer_enc,   Y_jouer )

Y_10   = NBC_10.predict(X_jouer_enc)
Y_05   = NBC_05.predict(X_jouer_enc)
Y_00   = NBC_00.predict(X_jouer_enc)


print(          'Alpha = 1.0'             )
print(classification_report(Y_jouer, Y_10))

print(          'Alpha = 0.5'             )
print(classification_report(Y_jouer, Y_05))

print(          'Alpha = 0.0'             )
print(classification_report(Y_jouer, Y_00))


Alpha = 1.0
              precision    recall  f1-score   support

         non       1.00      0.80      0.89         5
         oui       0.90      1.00      0.95         9

    accuracy                           0.93        14
   macro avg       0.95      0.90      0.92        14
weighted avg       0.94      0.93      0.93        14

Alpha = 0.5
              precision    recall  f1-score   support

         non       1.00      0.80      0.89         5
         oui       0.90      1.00      0.95         9

    accuracy                           0.93        14
   macro avg       0.95      0.90      0.92        14
weighted avg       0.94      0.93      0.93        14

Alpha = 0.0
              precision    recall  f1-score   support

         non       1.00      0.80      0.89         5
         oui       0.90      1.00      0.95         9

    accuracy                           0.93        14
   macro avg       0.95      0.90      0.92        14
weighted avg       0.94      0.93     

C:\Users\pc\anaconda3\lib\site-packages\sklearn\naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(


**TODO: Analyser les résultats**

- Que remarquez-vous ?
- Est-ce que le lissage affecte la performance dans ce cas ? Pourquoi ?
- Pourquoi Scikit-learn n'accepte pas la valeur $\alpha=0$ et affiche une alerte "UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10" ?
- Quelle est l'intérêt du lissage (dans le cas général) ?

**Réponse**

- En analysant les résultats, on remarque que les trois modèles donnent des performances similaires, avec une précision, un rappel et une mesure F1 élevés pour les deux classes ('non' et 'oui'), ainsi qu'une précision globale de 0,93.

- Le lissage n'a pas beaucoup d'effet sur la performance de ce modèle dans ce cas particulier. Cela peut être dû à plusieurs facteurs, dont la taille de l'ensemble de données. Comme cet ensemble de données est petit, il peut y avoir peu de risques de sur-ajustement et donc le lissage n'a pas un effet significatif sur la performance du modèle. De plus, le nombre de caractéristiques et de classes dans cet ensemble de données est également limité, ce qui réduit la complexité du modèle.

- Dans la formule de Lidstone, on divise le nombre d'occurrences de chaque catégorie par le nombre total d'occurrences, ce qui peut conduire à une division par zéro si le nombre d'occurrences pour une catégorie donnée est égal à zéro. Pour éviter cette erreur, on ajoute une petite quantité alpha à chaque compteur de catégorie. Si alpha est égal à zéro, alors la division est impossible car on divise par zéro. Pour éviter cette erreur, Scikit-learn utilise une valeur très petite pour alpha, telle que 1.0e-10, lorsqu'elle est proche de zéro. Cette valeur est généralement suffisamment petite pour éviter les erreurs numériques tout en garantissant que l'ajout de cette petite quantité ne modifie pas significativement les estimations de probabilité pour les différentes classes.

- En général, l'intérêt du lissage est de permettre une meilleure généralisation du modèle, en évitant que les estimations de probabilité pour certaines classes soient trop élevées ou trop basses, ce qui pourrait entraîner un sur-ajustement et une performance médiocre sur de nouvelles données. Le lissage aide également à éviter les cas où une caractéristique donnée n'est pas présente dans la classe d'apprentissage, ce qui conduirait à une estimation de probabilité nulle pour cette caractéristique. Dans ces cas-là, le lissage permet d'attribuer une petite probabilité à la caractéristique, évitant ainsi une estimation de probabilité nulle et améliorant la performance du modèle.

### II.3. Comparaison avec d'autres algorithmes

Naive Bayes est un algorithme puissant lorsqu'il s'agit de classer les documents textuels ; nous voulons tester cette information avec la détection de spam. 
Le dataset utilisé est [SMS Spam Collection Dataset](https://www.kaggle.com/uciml/sms-spam-collection-dataset).
Chaque message du dataset doit être représenté sous forme d'un modèle "Sac à mots" (BoW : Bag of Words).
Dans l'entraînement, les différents mots qui s'apparaissent dans les messages (vocabulaire) sont considérés comme des caractéristiques. 
Donc, pour chaque message, la valeur de la caractéristique est la fréquence du mot dans le message. 
Par exemple, si le mot "good" apparait 3 fois dans le message, donc la caractéristique "good" aura la valeur 3 dans ce message.

Notre implémentation n'est pas adéquate pour la nature de ce problème. 
Dans Scikit-learn, [sklearn.naive_bayes.CategoricalNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.CategoricalNB.html) est similaire à notre implémentation. 
L'algorithme adéquat pour ce type de problème est [sklearn.naive_bayes.MultinomialNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html).
Les algorithmes comparés :
1. Naive Bayes (Loi Multinomiale)
1. Naive Bayes (Loi Gaussienne)
1. Regression logistique 
1. Arbres de decision

In [153]:
# lire le dataset
messages = pd.read_csv('Desktop/2Sem/ML/TPs/TP04/TP04/data/spam.csv', encoding='latin-1')
# renomer les caractéristiques : texte et classe
messages = messages.rename(columns={'v1': 'classe', 'v2': 'texte'})
# garder seulement ces deux caractéristiques
messages = messages.filter(['texte', 'classe'])

messages.head()

,texte,classe
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham


In [154]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection         import train_test_split
from sklearn.naive_bayes             import MultinomialNB, GaussianNB
from sklearn.linear_model            import LogisticRegression
from sklearn.tree                    import DecisionTreeClassifier
from sklearn.metrics                 import precision_score, recall_score
import timeit


modeles = [
    MultinomialNB(),
    GaussianNB(),
    LogisticRegression(solver='lbfgs') ,
    #solver=sag est plus lent; donc j'ai choisi le plus rapide
    DecisionTreeClassifier()
]

temps_train = []
temps_test  = []
rappel      = []
precision   = []

msg_train, msg_test, Y_train, Y_test = train_test_split(messages['texte'] ,
                                                        messages['classe'],
                                                        test_size    = 0.2, 
                                                        random_state = 0  )

count_vectorizer = CountVectorizer()
X_train          = count_vectorizer.fit_transform(msg_train).toarray()
X_test           = count_vectorizer.transform    (msg_test ).toarray()


for modele in modeles:
    # ==================================
    # ENTRAINEMENT 
    # ==================================
    temps_debut = timeit.default_timer()
    modele.fit(X_train, Y_train)
    temps_train.append(timeit.default_timer() - temps_debut)
    
    # ==================================
    # TEST 
    # ==================================
    temps_debut = timeit.default_timer()
    Y_pred      = modele.predict(X_test)
    temps_test.append(timeit.default_timer() - temps_debut)
    
    # ==================================
    # PERFORMANCE 
    # ==================================
    # Ici, nous considérons une classification binaire avec une seule classe "spam" 
    # le classifieur ne sera pas jugé par sa capacité de détecter les non spams
    precision.append(precision_score(Y_test, Y_pred, pos_label='spam'))
    rappel   .append(recall_score   (Y_test, Y_pred, pos_label='spam'))

    
print('Fin') 

Fin


#### II.3.1. Temps d'entraînement et de test

Combien de temps chaque algorithme prend pour entrainer le même dataset d'entrainement et combien de temps pour tester le même dataset de test.

In [14]:
algo_noms = ['Naive Bayes Multinomial (NBM)', 
             'Naive Bayes Gaussien (NBG)'   , 
             'Regression logistique (RL)'   , 
             'Arbre de decision (AD)']

pd.DataFrame({
    'Algorithme'            : algo_noms  ,
    'Temps d\'entrainement' : temps_train,
    'Temps de test'         : temps_test
})

,Algorithme,Temps d'entrainement,Temps de test
0,Naive Bayes Multinomial (NBM),0.267185,0.018101
1,Naive Bayes Gaussien (NBG),0.318484,0.074441
2,Regression logistique (RL),0.836053,0.016207
3,Arbre de decision (AD),8.721873,0.010284


**TODO: Analyser les résultats**

- Que remarquez-vous concernant le temps d'entrainement ? (ordonner les algorithmes)
- Pourquoi nous avons eu ces résultats en se basant sur les algorithmes ? (discuter chaque algorithme vis-a-vis le temps d'entrainement)
- Que remarquez-vous concernant le temps de test ? (ordonner les algorithmes)
- Pourquoi nous avons eu ces résultats en se basant sur les algorithmes ? (discuter chaque algorithme vis-a-vis le temps de test)

**Réponse**

- En examinant les résultats, on peut remarquer que le temps d'entraînement varie considérablement selon l'algorithme utilisé. L'arbre de décision prend le plus de temps pour s'entraîner, suivi de la régression logistique, du Naive Bayes Gaussien et enfin du Naive Bayes Multinomial. Par conséquent, le classement de temps d'entraînement, du plus long au plus court, serait : arbre de décision, régression logistique, Naive Bayes Gaussien, Naive Bayes Multinomial.


- Ces résultats peuvent être expliqués en fonction de la complexité de l'algorithme. 
   - **Naive Bayes Multinomial** : L'entraînement du modèle Naive Bayes Multinomial est relativement simple et rapide, car il suffit de compter le nombre d'occurrences de chaque mot dans chaque message. La représentation "Sac à mots" est très efficace pour représenter les données textuelles, car elle ne tient pas compte de l'ordre des mots dans les phrases, mais seulement de leur fréquence. Cette représentation permet de réduire le temps d'entraînement, car le modèle est beaucoup moins complexe que d'autres modèles qui prennent en compte l'ordre des mots, tels que les réseaux de neurones.
   - **Naive Bayes Gaussien** : contrairement au Naive Bayes Multinomial, le Naive Bayes Gaussien suppose que les caractéristiques suivent une distribution normale. Cela signifie que pour chaque classe, le modèle doit estimer la moyenne et la variance de chaque caractéristique. Pour ce faire, l'algorithme doit parcourir tout le dataset plusieurs fois pour estimer ces paramètres. Par conséquent, le temps d'entrainement du Naive Bayes Gaussien est plus long que celui du Naive Bayes Multinomial.
   - **Regression logistique** : Cette méthode est également plus complexe que les deux précedentes car elle suppose une relation linéaire entre les caractéristiques et la variable cible. Pour entraîner le modèle, l'algorithme utilise une fonction de coût qui mesure l'erreur de prédiction du modèle et ajuste les coefficients pour minimiser cette erreur. Ce processus d'optimisation nécessite des calculs itératifs pour trouver les coefficients optimaux qui minimisent la fonction de coût.
   - **Arbre de décision** : Cet algorithme est plus complexe que les autres,La complexité de l'algorithme réside dans le processus "itératif" de construction de l'arbre. L'algorithme doit sélectionner les caractéristiques qui permettent de séparer au mieux les données en fonction de la variable cible en utilisant une fonction de mesure d'impurté comme l'entropie ou le coefficient de Gini.

- En ce qui concerne les temps d'exécution des tests, le classement est différent. Le temps de test le plus court est pour l'arbre de décision, suivi de la régression logistique, du Naive Bayes Multinomial et du Naive Bayes Gaussien. Par conséquent, le classement de temps de test, du plus court au plus long, serait : arbre de décision, régression logistique, Naive Bayes Multinomial, Naive Bayes Gaussien.


- En ce qui concerne le temps de test, les algorithmes qui ont produit les résultats les plus rapides ont tendance à être ceux qui sont les plus spécialisés pour les données de texte, comme l'arbre de décision et la régression logistique. Les Naive Bayes sont également assez rapides à tester car leur calcul de probabilité est simple, mais le Naive Bayes Gaussien a tendance à être plus lent car il doit estimer la moyenne et la variance pour chaque caractéristique.
    - **Arbre de décision** :  même si la construction de l'arbre de décision peut prendre du temps pendant l'entraînement. Cependant, lors des tests, le temps d'exécution de l'arbre de décision est plus court que celui des autres algorithmes. Cela est dû au fait que pour chaque message, l'arbre de décision effectue une seule descente dans l'arbre pour déterminer la classe prédite, ce qui est généralement plus rapide que les calculs nécessaires pour les autres algorithmes.
    - **Régression Logistique**: La raison pour laquelle la régression logistique est relativement rapide pour les tests est que les prédictions sont basées sur une simple multiplication matricielle et une addition, ce qui est un calcul rapide. Une fois que les coefficients optimaux ont été trouvés pendant l'entraînement (implique de résoudre un système dequations linéaires qui necessite des calculs itératifs pour converger) , alors que les tests sont simplement une question de multiplication matricielle et d'addition.
    - **Naive Bayes Multinomial**: lors du test, le modèle doit prédire la classe pour chaque nouveau message en calculant la probabilité de chaque classe pour ce message. Cela implique de calculer les probabilités conditionnelles pour chaque mot dans le message, ce qui peut être coûteux en termes de temps de calcul, surtout si le nombre de caractéristiques (mots) est élevé. Par conséquent, le Naive Bayes Multinomial peut être lent lors des tests, même s'il est rapide lors de l'entraînement.
    - **Naive Bayes Gaussien**: Le Naive Bayes Gaussien peut devenir très lent lors des tests car il doit estimer la moyenne et la variance pour chaque caractéristique de l'ensemble de test. Cela peut être coûteux en termes de temps de calcul, surtout si le nombre de caractéristiques est important. De plus, le Naive Bayes Gaussien suppose que les données suivent une distribution normale, ce qui peut ne pas être vrai pour toutes les caractéristiques. Dans ce cas, les estimations des moyennes et des variances peuvent ne pas être précises, ce qui peut également affecter la performance de l'algorithme lors des tests.








#### II.3.2. Qualité de prédiction

Comment chaque algorithme performe sur le dataset de test dans le cas de détection de spams (spam: est la classe positive).

In [15]:
pd.DataFrame({
    'Algorithme' : algo_noms,
    'Rappel'     : rappel   ,
    'Precision'  : precision
})

,Algorithme,Rappel,Precision
0,Naive Bayes Multinomial (NBM),0.927711,0.987179
1,Naive Bayes Gaussien (NBG),0.891566,0.616667
2,Regression logistique (RL),0.855422,0.986111
3,Arbre de decision (AD),0.873494,0.895062


**TODO: Analyser les résultats**

On remarque que Naive Bayes surpasse la régression logistique pour la détection de spams. 
- Est-ce que ceci preuve que Naive Bayes est meilleur que les autres algorithmes sur n'importe quel problème ?
- Est-ce que ceci preuve que Naive Bayes peut donner de meilleurs résultats que les autres algorithmes sur des problèmes similaires ?
- Pourquoi le modèle gaussien est moins performant que le multinomial en se basant sur la nature des deux algorithmes ?
- Pourquoi le modèle gaussien est moins performant que le multinomial en se basant sur la nature du probleme/donnees ?

**Réponse**

- Non. La supériorité du Naive Bayes sur la régression logistique dans la détection de spams ne prouve pas nécessairement que le Naive Bayes est meilleur que les autres algorithmes sur n'importe quel problème. Les performances d'un algorithme dépendent de la nature des données et du problème à résoudre.


- Non. Les résultats obtenus dans cette étude montrent que Naive  a produit des performances supérieures à celles des autres algorithmes pour la tâche de détection de spam. Cependant, cela ne signifie pas que Naive Bayes sera toujours le meilleur algorithme pour tous les problèmes similaires. Les performances des algorithmes dépendent de nombreux facteurs tels que la nature des données, les paramètres d'entrée, la taille de l'ensemble de données.


- Le modèle gaussien est moins performant que le multinomial car les données textuelles ont une distribution multinomiale, ce qui signifie que chaque caractéristique peut prendre un nombre fini de valeurs discrètes. Le modèle gaussien, quant à lui, suppose que les caractéristiques ont une distribution normale (ou gaussienne). Cette hypothèse n'est pas toujours valide pour les données textuelles, ce qui peut affecter négativement les performances du modèle gaussien.

    De plus, le modèle gaussien nécessite l'estimation de la moyenne et de la variance pour chaque caractéristique, ce qui peut être coûteux en termes de temps de traitement et de mémoire. En revanche, le modèle multinomial est plus simple et plus rapide à calculer car il ne nécessite que le compte des occurrences de chaque mot dans chaque document, ce qui est plus approprié pour les données textuelles.
    En ce qui concerne la nature des deux algorithmes, le modèle multinomial est mieux adapté aux données textuelles car il peut prendre en compte la fréquence d'apparition des mots dans les documents, alors que le modèle gaussien suppose que les caractéristiques sont continues et ont une distribution normale, ce qui peut ne pas être le cas pour les données textuelles.
    
    
- Le modèle gaussien est moins performant que le multinomial dans ce cas particulier en raison de la nature des données de spam. Les caractéristiques utilisées pour décrire les courriels (par exemple, la fréquence des mots) sont mieux décrites par une distribution multinomiale, car elles décrivent la fréquence d'occurrence de chaque mot dans un texte. En revanche, le modèle gaussien suppose que les données sont distribuées selon une loi normale, ce qui peut ne pas être le cas pour les caractéristiques des courriels de spam. Par conséquent, le modèle gaussien peut ne pas être en mesure de modéliser correctement la distribution des données, ce qui peut entraîner des performances inférieures.

In [16]:
print("  _____    __                                              _               ")
print(" |_   _|  / _|                                            | |              ")
print("   | |   | |_     _   _    ___    _   _      __ _    ___  | |_             ")
print("   | |   |  _|   | | | |  / _ \  | | | |    / _` |  / _ \ | __|            ")
print("  _| |_  | |     | |_| | | (_) | | |_| |   | (_| | |  __/ | |_             ")
print(" |_____| |_|      \__, |  \___/   \__,_|    \__, |  \___|  \__|            ")
print("                   __/ |                     __/ |                         ")
print("                  |___/                     |___/                          ")
print("  _     _       _            __                                            ")
print(" | |   | |     (_)          / _|                 _                         ")
print(" | |_  | |__    _   ___    | |_    __ _   _ __  (_)                        ")
print(" | __| | '_ \  | | / __|   |  _|  / _` | | '__|                            ")
print(" | |_  | | | | | | \__ \   | |   | (_| | | |     _                         ")
print("  \__| |_| |_| |_| |___/   |_|    \__,_| |_|    ( )                        ")
print("                                                |/                         ")
print("                                                                           ")
print("                                                                           ")
print("                                                                           ")
print("  _   _    ___    _   _      __ _   _ __    ___                            ")
print(" | | | |  / _ \  | | | |    / _` | | '__|  / _ \                           ")
print(" | |_| | | (_) | | |_| |   | (_| | | |    |  __/                           ")
print("  \__, |  \___/   \__,_|    \__,_| |_|     \___|                           ")
print("   __/ |                                                                   ")
print("  |___/                                                                    ")
print("                    _                                                __    ")
print("                   | |                                            _  \ \   ")
print("  _ __     ___     | |__    _   _   _ __ ___     __ _   _ __     (_)  | |  ")
print(" | '_ \   / _ \    | '_ \  | | | | | '_ ` _ \   / _` | | '_ \         | |  ")
print(" | | | | | (_) |   | | | | | |_| | | | | | | | | (_| | | | | |    _   | |  ")
print(" |_| |_|  \___/    |_| |_|  \__,_| |_| |_| |_|  \__,_| |_| |_|   (_)  | |  ")
print("                                                                     /_/   ")
print("                                                                           ")

  _____    __                                              _               
 |_   _|  / _|                                            | |              
   | |   | |_     _   _    ___    _   _      __ _    ___  | |_             
   | |   |  _|   | | | |  / _ \  | | | |    / _` |  / _ \ | __|            
  _| |_  | |     | |_| | | (_) | | |_| |   | (_| | |  __/ | |_             
 |_____| |_|      \__, |  \___/   \__,_|    \__, |  \___|  \__|            
                   __/ |                     __/ |                         
                  |___/                     |___/                          
  _     _       _            __                                            
 | |   | |     (_)          / _|                 _                         
 | |_  | |__    _   ___    | |_    __ _   _ __  (_)                        
 | __| | '_ \  | | / __|   |  _|  / _` | | '__|                            
 | |_  | | | | | | \__ \   | |   | (_| | | |     _                         
  \__| |_| |